In [1]:
import os
import glob
import numpy as np
from PIL import Image

In [2]:
train_irrg_dir = "/media/syz/Remote_Sensing/dataset/potsdam/IRRG_Images_for_participants_no_Bounary"
train_irrg_label_dir = "/media/syz/Remote_Sensing/dataset/potsdam/Labels_for_participants_no_Boundary"

test_irrg_dir = "/media/syz/Remote_Sensing/dataset/potsdam/IRRG_Images_for_test_no_Bounary"
test_irrg_label_dir = "/media/syz/Remote_Sensing/dataset/potsdam/Labels_for_test_no_Boundary"

In [3]:
train_irrg_list = glob.glob(train_irrg_dir+"/*.tif")
train_irrg_list = sorted(train_irrg_list)
train_irrg_list[-5:]

['/media/syz/Remote_Sensing/dataset/potsdam/IRRG_Images_for_participants_no_Bounary/top_potsdam_7_11_IRRG.tif',
 '/media/syz/Remote_Sensing/dataset/potsdam/IRRG_Images_for_participants_no_Bounary/top_potsdam_7_12_IRRG.tif',
 '/media/syz/Remote_Sensing/dataset/potsdam/IRRG_Images_for_participants_no_Bounary/top_potsdam_7_7_IRRG.tif',
 '/media/syz/Remote_Sensing/dataset/potsdam/IRRG_Images_for_participants_no_Bounary/top_potsdam_7_8_IRRG.tif',
 '/media/syz/Remote_Sensing/dataset/potsdam/IRRG_Images_for_participants_no_Bounary/top_potsdam_7_9_IRRG.tif']

In [4]:
train_irrg_label_list = glob.glob(train_irrg_label_dir+"/*.tif")
train_irrg_label_list = sorted(train_irrg_label_list)
train_irrg_label_list[-5:]

['/media/syz/Remote_Sensing/dataset/potsdam/Labels_for_participants_no_Boundary/top_potsdam_7_11_label_noBoundary.tif',
 '/media/syz/Remote_Sensing/dataset/potsdam/Labels_for_participants_no_Boundary/top_potsdam_7_12_label_noBoundary.tif',
 '/media/syz/Remote_Sensing/dataset/potsdam/Labels_for_participants_no_Boundary/top_potsdam_7_7_label_noBoundary.tif',
 '/media/syz/Remote_Sensing/dataset/potsdam/Labels_for_participants_no_Boundary/top_potsdam_7_8_label_noBoundary.tif',
 '/media/syz/Remote_Sensing/dataset/potsdam/Labels_for_participants_no_Boundary/top_potsdam_7_9_label_noBoundary.tif']

In [5]:
test_irrg_list = glob.glob(test_irrg_dir+"/*.tif")
test_irrg_list = sorted(test_irrg_list)
test_irrg_list[-5:]

['/media/syz/Remote_Sensing/dataset/potsdam/IRRG_Images_for_test_no_Bounary/top_potsdam_5_15_IRRG.tif',
 '/media/syz/Remote_Sensing/dataset/potsdam/IRRG_Images_for_test_no_Bounary/top_potsdam_6_13_IRRG.tif',
 '/media/syz/Remote_Sensing/dataset/potsdam/IRRG_Images_for_test_no_Bounary/top_potsdam_6_14_IRRG.tif',
 '/media/syz/Remote_Sensing/dataset/potsdam/IRRG_Images_for_test_no_Bounary/top_potsdam_6_15_IRRG.tif',
 '/media/syz/Remote_Sensing/dataset/potsdam/IRRG_Images_for_test_no_Bounary/top_potsdam_7_13_IRRG.tif']

In [6]:
test_irrg_label_list = glob.glob(test_irrg_label_dir+"/*.tif")
test_irrg_label_list = sorted(test_irrg_label_list)
test_irrg_label_list[-5:]

['/media/syz/Remote_Sensing/dataset/potsdam/Labels_for_test_no_Boundary/top_potsdam_5_15_label_noBoundary.tif',
 '/media/syz/Remote_Sensing/dataset/potsdam/Labels_for_test_no_Boundary/top_potsdam_6_13_label_noBoundary.tif',
 '/media/syz/Remote_Sensing/dataset/potsdam/Labels_for_test_no_Boundary/top_potsdam_6_14_label_noBoundary.tif',
 '/media/syz/Remote_Sensing/dataset/potsdam/Labels_for_test_no_Boundary/top_potsdam_6_15_label_noBoundary.tif',
 '/media/syz/Remote_Sensing/dataset/potsdam/Labels_for_test_no_Boundary/top_potsdam_7_13_label_noBoundary.tif']

In [7]:
patch_size = 512
overlap_rate = 1/3
overlap = int(512*(1/3))
print(f"patch_size: {patch_size}, overlap_rate: {overlap_rate:.5f}, overlap: {overlap}")

patch_size: 512, overlap_rate: 0.33333, overlap: 170


In [13]:
save_train_image_dir = "/media/syz/Remote_Sensing/dataset/potsdam/Patches/Patch_IRRG_for_participants_no_Boundary"
save_train_label_dir = "/media/syz/Remote_Sensing/dataset/potsdam/Patches/Patch_IRRG_label_for_participants_no_Boundary"

save_test_image_dir = "/media/syz/Remote_Sensing/dataset/potsdam/Patches/Patch_IRRG_for_test_no_Boundary"
save_test_label_dir = "/media/syz/Remote_Sensing/dataset/potsdam/Patches/Patch_IRRG_label_for_test_no_Boundary"

# Extract Patch from Train Datset (Overlap)

In [100]:
def extract_train_patches(image_list, label_list, save_dest_for_image, save_dest_for_label, patch_size, overlap_rate):
    
    overlap = int(patch_size*(1-overlap_rate)) # Most Important: overlap based on <patch_size>. 

    for idx, (img_path, label_path) in enumerate(zip(image_list, label_list)):
        img = Image.open(img_path)
        label = Image.open(label_path)
        assert img.size == label.size, f" Different Shape Image and Label. "
        (height, width) = img.size
        img_dir = os.path.basename(img_path).split('.')[0]
        label_dir = os.path.basename(label_path).split('.')[0]

        # extract patch start
        h = 0
        height_count=0
        extract_finish = False

        while not extract_finish:

            if h+patch_size <= height:

                # set row flag
                extract_row = True
                # initial row start
                w = 0
                width_count = 0
                # Extracting row
                while extract_row:
                    if w+patch_size <= width:
                        print(f"h:{h} w:{w} ->  h':{h+patch_size} w':{w+patch_size}")
                        patch_img = img.crop([h, w, h+patch_size, w+patch_size])
                        patch_label = label.crop([h, w, h+patch_size, w+patch_size])
                    else:
                        print(f"h:{h} w:{width-patch_size} ->  h':{h+patch_size} w':{width}")                        
                        patch_img = img.crop([h, width-patch_size, h+patch_size, width])
                        patch_label = label.crop([h, width-patch_size, h+patch_size, width])
                        # stop
                        extract_row = False

                    if not os.path.exists(os.path.join(save_dest_for_image, img_dir)):
                        os.mkdir(os.path.join(save_dest_for_image, img_dir))
                    if not os.path.exists(os.path.join(save_dest_for_label, label_dir)):
                        os.mkdir(os.path.join(save_dest_for_label, label_dir))

                    patch_img_name = img_dir+'_'+str(height_count)+'_'+str(width_count)+'.png'
                    patch_label_name = label_dir+'_'+str(height_count)+'_'+str(width_count)+'.png'

                    patch_img.save(os.path.join(save_dest_for_image, img_dir, patch_img_name))
                    patch_label.save(os.path.join(save_dest_for_label, label_dir, patch_label_name))

                    # iter along w
                    w += overlap
                    width_count += 1
                    #
                    print(height_count, width_count)

            else:
                # set last row flag
                extract_last_row = True
                # initial row start
                w = 0
                width_count = 0
                # Extracting Last row
                while extract_last_row:

                    if w+patch_size <= width:
                        print(f"h:{height-patch_size} w:{w} ->  h':{height} w':{w+patch_size}")
                        patch_img = img.crop([height-patch_size, w, height, w+patch_size])
                        patch_label = label.crop([height-patch_size, w, height, w+patch_size])
                    else:
                        print(f"h:{height-patch_size} w:{w-patch_size} ->  h':{height} w':{width}")
                        patch_img = img.crop([height-patch_size, width-patch_size, height, width])
                        patch_label = label.crop([height-patch_size, width-patch_size, height, width])
                        # stop
                        extract_last_row = False
                        extract_finish = True

                    if not os.path.exists(os.path.join(save_dest_for_image, img_dir)):
                        os.mkdir(os.path.join(save_dest_for_image, img_dir))
                    if not os.path.exists(os.path.join(save_dest_for_label, label_dir)):
                        os.mkdir(os.path.join(save_dest_for_label, label_dir))

                    patch_img_name = img_dir+'_'+str(height_count)+'_'+str(width_count)+'.png'
                    patch_label_name = label_dir+'_'+str(height_count)+'_'+str(width_count)+'.png'

                    patch_img.save(os.path.join(save_dest_for_image, img_dir, patch_img_name))
                    patch_label.save(os.path.join(save_dest_for_label, label_dir, patch_label_name))
                    
                    # iter along w
                    w += overlap
                    width_count += 1
                    #
                    print(height_count, width_count)
                    
            # iter along h
            h += overlap
            height_count += 1

        # extract patch end
        print(f"Image {img_dir} extracted finished !")

In [101]:
extract_train_patches(
    train_irrg_list, 
    train_irrg_label_list, 
    save_dest_for_image=save_train_image_dir, 
    save_dest_for_label=save_train_label_dir, 
    patch_size=512, 
    overlap_rate=1/3)

[0, 0, 512, 512]
0 1
[0, 341, 512, 853]
0 2
[0, 682, 512, 1194]
0 3
[0, 1023, 512, 1535]
0 4
[0, 1364, 512, 1876]
0 5
[0, 1705, 512, 2217]
0 6
[0, 2046, 512, 2558]
0 7
[0, 2387, 512, 2899]
0 8
[0, 2728, 512, 3240]
0 9
[0, 3069, 512, 3581]
0 10
[0, 3410, 512, 3922]
0 11
[0, 3751, 512, 4263]
0 12
[0, 4092, 512, 4604]
0 13
[0, 4433, 512, 4945]
0 14
[0, 4774, 512, 5286]
0 15
[0, 5115, 512, 5627]
0 16
[0, 5456, 512, 5968]
0 17
[0, 5488, 512, 6000]
0 18
[341, 0, 853, 512]
1 1
[341, 341, 853, 853]
1 2
[341, 682, 853, 1194]
1 3
[341, 1023, 853, 1535]
1 4
[341, 1364, 853, 1876]
1 5
[341, 1705, 853, 2217]
1 6
[341, 2046, 853, 2558]
1 7
[341, 2387, 853, 2899]
1 8
[341, 2728, 853, 3240]
1 9
[341, 3069, 853, 3581]
1 10
[341, 3410, 853, 3922]
1 11
[341, 3751, 853, 4263]
1 12
[341, 4092, 853, 4604]
1 13
[341, 4433, 853, 4945]
1 14
[341, 4774, 853, 5286]
1 15
[341, 5115, 853, 5627]
1 16
[341, 5456, 853, 5968]
1 17
[341, 5488, 853, 6000]
1 18
[682, 0, 1194, 512]
2 1
[682, 341, 1194, 853]
2 2
[682, 682,

15 13
[5115, 4433, 5627, 4945]
15 14
[5115, 4774, 5627, 5286]
15 15
[5115, 5115, 5627, 5627]
15 16
[5115, 5456, 5627, 5968]
15 17
[5115, 5488, 5627, 6000]
15 18
[5456, 0, 5968, 512]
16 1
[5456, 341, 5968, 853]
16 2
[5456, 682, 5968, 1194]
16 3
[5456, 1023, 5968, 1535]
16 4
[5456, 1364, 5968, 1876]
16 5
[5456, 1705, 5968, 2217]
16 6
[5456, 2046, 5968, 2558]
16 7
[5456, 2387, 5968, 2899]
16 8
[5456, 2728, 5968, 3240]
16 9
[5456, 3069, 5968, 3581]
16 10
[5456, 3410, 5968, 3922]
16 11
[5456, 3751, 5968, 4263]
16 12
[5456, 4092, 5968, 4604]
16 13
[5456, 4433, 5968, 4945]
16 14
[5456, 4774, 5968, 5286]
16 15
[5456, 5115, 5968, 5627]
16 16
[5456, 5456, 5968, 5968]
16 17
[5456, 5488, 5968, 6000]
16 18
[5488, 0, 6000, 512]
17 1
[5488, 341, 6000, 853]
17 2
[5488, 682, 6000, 1194]
17 3
[5488, 1023, 6000, 1535]
17 4
[5488, 1364, 6000, 1876]
17 5
[5488, 1705, 6000, 2217]
17 6
[5488, 2046, 6000, 2558]
17 7
[5488, 2387, 6000, 2899]
17 8
[5488, 2728, 6000, 3240]
17 9
[5488, 3069, 6000, 3581]
17 10
[54

13 6
[4433, 2046, 4945, 2558]
13 7
[4433, 2387, 4945, 2899]
13 8
[4433, 2728, 4945, 3240]
13 9
[4433, 3069, 4945, 3581]
13 10
[4433, 3410, 4945, 3922]
13 11
[4433, 3751, 4945, 4263]
13 12
[4433, 4092, 4945, 4604]
13 13
[4433, 4433, 4945, 4945]
13 14
[4433, 4774, 4945, 5286]
13 15
[4433, 5115, 4945, 5627]
13 16
[4433, 5456, 4945, 5968]
13 17
[4433, 5488, 4945, 6000]
13 18
[4774, 0, 5286, 512]
14 1
[4774, 341, 5286, 853]
14 2
[4774, 682, 5286, 1194]
14 3
[4774, 1023, 5286, 1535]
14 4
[4774, 1364, 5286, 1876]
14 5
[4774, 1705, 5286, 2217]
14 6
[4774, 2046, 5286, 2558]
14 7
[4774, 2387, 5286, 2899]
14 8
[4774, 2728, 5286, 3240]
14 9
[4774, 3069, 5286, 3581]
14 10
[4774, 3410, 5286, 3922]
14 11
[4774, 3751, 5286, 4263]
14 12
[4774, 4092, 5286, 4604]
14 13
[4774, 4433, 5286, 4945]
14 14
[4774, 4774, 5286, 5286]
14 15
[4774, 5115, 5286, 5627]
14 16
[4774, 5456, 5286, 5968]
14 17
[4774, 5488, 5286, 6000]
14 18
[5115, 0, 5627, 512]
15 1
[5115, 341, 5627, 853]
15 2
[5115, 682, 5627, 1194]
15 3
[

10 18
[3751, 0, 4263, 512]
11 1
[3751, 341, 4263, 853]
11 2
[3751, 682, 4263, 1194]
11 3
[3751, 1023, 4263, 1535]
11 4
[3751, 1364, 4263, 1876]
11 5
[3751, 1705, 4263, 2217]
11 6
[3751, 2046, 4263, 2558]
11 7
[3751, 2387, 4263, 2899]
11 8
[3751, 2728, 4263, 3240]
11 9
[3751, 3069, 4263, 3581]
11 10
[3751, 3410, 4263, 3922]
11 11
[3751, 3751, 4263, 4263]
11 12
[3751, 4092, 4263, 4604]
11 13
[3751, 4433, 4263, 4945]
11 14
[3751, 4774, 4263, 5286]
11 15
[3751, 5115, 4263, 5627]
11 16
[3751, 5456, 4263, 5968]
11 17
[3751, 5488, 4263, 6000]
11 18
[4092, 0, 4604, 512]
12 1
[4092, 341, 4604, 853]
12 2
[4092, 682, 4604, 1194]
12 3
[4092, 1023, 4604, 1535]
12 4
[4092, 1364, 4604, 1876]
12 5
[4092, 1705, 4604, 2217]
12 6
[4092, 2046, 4604, 2558]
12 7
[4092, 2387, 4604, 2899]
12 8
[4092, 2728, 4604, 3240]
12 9
[4092, 3069, 4604, 3581]
12 10
[4092, 3410, 4604, 3922]
12 11
[4092, 3751, 4604, 4263]
12 12
[4092, 4092, 4604, 4604]
12 13
[4092, 4433, 4604, 4945]
12 14
[4092, 4774, 4604, 5286]
12 15
[40

8 11
[2728, 3751, 3240, 4263]
8 12
[2728, 4092, 3240, 4604]
8 13
[2728, 4433, 3240, 4945]
8 14
[2728, 4774, 3240, 5286]
8 15
[2728, 5115, 3240, 5627]
8 16
[2728, 5456, 3240, 5968]
8 17
[2728, 5488, 3240, 6000]
8 18
[3069, 0, 3581, 512]
9 1
[3069, 341, 3581, 853]
9 2
[3069, 682, 3581, 1194]
9 3
[3069, 1023, 3581, 1535]
9 4
[3069, 1364, 3581, 1876]
9 5
[3069, 1705, 3581, 2217]
9 6
[3069, 2046, 3581, 2558]
9 7
[3069, 2387, 3581, 2899]
9 8
[3069, 2728, 3581, 3240]
9 9
[3069, 3069, 3581, 3581]
9 10
[3069, 3410, 3581, 3922]
9 11
[3069, 3751, 3581, 4263]
9 12
[3069, 4092, 3581, 4604]
9 13
[3069, 4433, 3581, 4945]
9 14
[3069, 4774, 3581, 5286]
9 15
[3069, 5115, 3581, 5627]
9 16
[3069, 5456, 3581, 5968]
9 17
[3069, 5488, 3581, 6000]
9 18
[3410, 0, 3922, 512]
10 1
[3410, 341, 3922, 853]
10 2
[3410, 682, 3922, 1194]
10 3
[3410, 1023, 3922, 1535]
10 4
[3410, 1364, 3922, 1876]
10 5
[3410, 1705, 3922, 2217]
10 6
[3410, 2046, 3922, 2558]
10 7
[3410, 2387, 3922, 2899]
10 8
[3410, 2728, 3922, 3240]
10 

...

16 9
[5456, 3069, 5968, 3581]
16 10
[5456, 3410, 5968, 3922]
16 11
[5456, 3751, 5968, 4263]
16 12
[5456, 4092, 5968, 4604]
16 13
[5456, 4433, 5968, 4945]
16 14
[5456, 4774, 5968, 5286]
16 15
[5456, 5115, 5968, 5627]
16 16
[5456, 5456, 5968, 5968]
16 17
[5456, 5488, 5968, 6000]
16 18
[5488, 0, 6000, 512]
17 1
[5488, 341, 6000, 853]
17 2
[5488, 682, 6000, 1194]
17 3
[5488, 1023, 6000, 1535]
17 4
[5488, 1364, 6000, 1876]
17 5
[5488, 1705, 6000, 2217]
17 6
[5488, 2046, 6000, 2558]
17 7
[5488, 2387, 6000, 2899]
17 8
[5488, 2728, 6000, 3240]
17 9
[5488, 3069, 6000, 3581]
17 10
[5488, 3410, 6000, 3922]
17 11
[5488, 3751, 6000, 4263]
17 12
[5488, 4092, 6000, 4604]
17 13
[5488, 4433, 6000, 4945]
17 14
[5488, 4774, 6000, 5286]
17 15
[5488, 5115, 6000, 5627]
17 16
[5488, 5456, 6000, 5968]
17 17
[5488, 5488, 6000, 6000]
17 18
Image top_potsdam_7_11_IRRG extracted finished !
[0, 0, 512, 512]
0 1
[0, 341, 512, 853]
0 2
[0, 682, 512, 1194]
0 3
[0, 1023, 512, 1535]
0 4
[0, 1364, 512, 1876]
0 5
[0, 170

14 3
[4774, 1023, 5286, 1535]
14 4
[4774, 1364, 5286, 1876]
14 5
[4774, 1705, 5286, 2217]
14 6
[4774, 2046, 5286, 2558]
14 7
[4774, 2387, 5286, 2899]
14 8
[4774, 2728, 5286, 3240]
14 9
[4774, 3069, 5286, 3581]
14 10
[4774, 3410, 5286, 3922]
14 11
[4774, 3751, 5286, 4263]
14 12
[4774, 4092, 5286, 4604]
14 13
[4774, 4433, 5286, 4945]
14 14
[4774, 4774, 5286, 5286]
14 15
[4774, 5115, 5286, 5627]
14 16
[4774, 5456, 5286, 5968]
14 17
[4774, 5488, 5286, 6000]
14 18
[5115, 0, 5627, 512]
15 1
[5115, 341, 5627, 853]
15 2
[5115, 682, 5627, 1194]
15 3
[5115, 1023, 5627, 1535]
15 4
[5115, 1364, 5627, 1876]
15 5
[5115, 1705, 5627, 2217]
15 6
[5115, 2046, 5627, 2558]
15 7
[5115, 2387, 5627, 2899]
15 8
[5115, 2728, 5627, 3240]
15 9
[5115, 3069, 5627, 3581]
15 10
[5115, 3410, 5627, 3922]
15 11
[5115, 3751, 5627, 4263]
15 12
[5115, 4092, 5627, 4604]
15 13
[5115, 4433, 5627, 4945]
15 14
[5115, 4774, 5627, 5286]
15 15
[5115, 5115, 5627, 5627]
15 16
[5115, 5456, 5627, 5968]
15 17
[5115, 5488, 5627, 6000]


11 15
[3751, 5115, 4263, 5627]
11 16
[3751, 5456, 4263, 5968]
11 17
[3751, 5488, 4263, 6000]
11 18
[4092, 0, 4604, 512]
12 1
[4092, 341, 4604, 853]
12 2
[4092, 682, 4604, 1194]
12 3
[4092, 1023, 4604, 1535]
12 4
[4092, 1364, 4604, 1876]
12 5
[4092, 1705, 4604, 2217]
12 6
[4092, 2046, 4604, 2558]
12 7
[4092, 2387, 4604, 2899]
12 8
[4092, 2728, 4604, 3240]
12 9
[4092, 3069, 4604, 3581]
12 10
[4092, 3410, 4604, 3922]
12 11
[4092, 3751, 4604, 4263]
12 12
[4092, 4092, 4604, 4604]
12 13
[4092, 4433, 4604, 4945]
12 14
[4092, 4774, 4604, 5286]
12 15
[4092, 5115, 4604, 5627]
12 16
[4092, 5456, 4604, 5968]
12 17
[4092, 5488, 4604, 6000]
12 18
[4433, 0, 4945, 512]
13 1
[4433, 341, 4945, 853]
13 2
[4433, 682, 4945, 1194]
13 3
[4433, 1023, 4945, 1535]
13 4
[4433, 1364, 4945, 1876]
13 5
[4433, 1705, 4945, 2217]
13 6
[4433, 2046, 4945, 2558]
13 7
[4433, 2387, 4945, 2899]
13 8
[4433, 2728, 4945, 3240]
13 9
[4433, 3069, 4945, 3581]
13 10
[4433, 3410, 4945, 3922]
13 11
[4433, 3751, 4945, 4263]
13 12
[44

9 9
[3069, 3069, 3581, 3581]
9 10
[3069, 3410, 3581, 3922]
9 11
[3069, 3751, 3581, 4263]
9 12
[3069, 4092, 3581, 4604]
9 13
[3069, 4433, 3581, 4945]
9 14
[3069, 4774, 3581, 5286]
9 15
[3069, 5115, 3581, 5627]
9 16
[3069, 5456, 3581, 5968]
9 17
[3069, 5488, 3581, 6000]
9 18
[3410, 0, 3922, 512]
10 1
[3410, 341, 3922, 853]
10 2
[3410, 682, 3922, 1194]
10 3
[3410, 1023, 3922, 1535]
10 4
[3410, 1364, 3922, 1876]
10 5
[3410, 1705, 3922, 2217]
10 6
[3410, 2046, 3922, 2558]
10 7
[3410, 2387, 3922, 2899]
10 8
[3410, 2728, 3922, 3240]
10 9
[3410, 3069, 3922, 3581]
10 10
[3410, 3410, 3922, 3922]
10 11
[3410, 3751, 3922, 4263]
10 12
[3410, 4092, 3922, 4604]
10 13
[3410, 4433, 3922, 4945]
10 14
[3410, 4774, 3922, 5286]
10 15
[3410, 5115, 3922, 5627]
10 16
[3410, 5456, 3922, 5968]
10 17
[3410, 5488, 3922, 6000]
10 18
[3751, 0, 4263, 512]
11 1
[3751, 341, 4263, 853]
11 2
[3751, 682, 4263, 1194]
11 3
[3751, 1023, 4263, 1535]
11 4
[3751, 1364, 4263, 1876]
11 5
[3751, 1705, 4263, 2217]
11 6
[3751, 2046

7 1
[2387, 341, 2899, 853]
7 2
[2387, 682, 2899, 1194]
7 3
[2387, 1023, 2899, 1535]
7 4
[2387, 1364, 2899, 1876]
7 5
[2387, 1705, 2899, 2217]
7 6
[2387, 2046, 2899, 2558]
7 7
[2387, 2387, 2899, 2899]
7 8
[2387, 2728, 2899, 3240]
7 9
[2387, 3069, 2899, 3581]
7 10
[2387, 3410, 2899, 3922]
7 11
[2387, 3751, 2899, 4263]
7 12
[2387, 4092, 2899, 4604]
7 13
[2387, 4433, 2899, 4945]
7 14
[2387, 4774, 2899, 5286]
7 15
[2387, 5115, 2899, 5627]
7 16
[2387, 5456, 2899, 5968]
7 17
[2387, 5488, 2899, 6000]
7 18
[2728, 0, 3240, 512]
8 1
[2728, 341, 3240, 853]
8 2
[2728, 682, 3240, 1194]
8 3
[2728, 1023, 3240, 1535]
8 4
[2728, 1364, 3240, 1876]
8 5
[2728, 1705, 3240, 2217]
8 6
[2728, 2046, 3240, 2558]
8 7
[2728, 2387, 3240, 2899]
8 8
[2728, 2728, 3240, 3240]
8 9
[2728, 3069, 3240, 3581]
8 10
[2728, 3410, 3240, 3922]
8 11
[2728, 3751, 3240, 4263]
8 12
[2728, 4092, 3240, 4604]
8 13
[2728, 4433, 3240, 4945]
8 14
[2728, 4774, 3240, 5286]
8 15
[2728, 5115, 3240, 5627]
8 16
[2728, 5456, 3240, 5968]
8 17
[27

# Extract Patch from Test Dataset (No Overlap)

In [59]:
# No Overlap
def extract_test_patches(image_list, label_list, save_dest_for_image, save_dest_for_label, patch_size,):
    
    for idx, (img_path, label_path) in enumerate(zip(image_list, label_list)):
        img = Image.open(img_path)
        label = Image.open(label_path)
        assert img.size == label.size, f" Different Shape Image and Label. "
        (height, width) = img.size
        img_dir = os.path.basename(img_path).split('.')[0]
        label_dir = os.path.basename(label_path).split('.')[0]

        # extract patch start
        h = 0
        height_count=0
        extract_finish = False

        while not extract_finish:

            if h+patch_size > height:
                # set last row flag
                extract_last_row = True
                # initial row start
                w = 0
                width_count = 0
                # Extracting Last row
                while extract_last_row:
            
                    if w + patch_size > width:
                        print(f"h:{height-patch_size} w:{width-patch_size} ->  h':{height} w':{width}")
                        patch_img = img.crop([height-patch_size, width-patch_size, height, width])
                        patch_label = label.crop([height-patch_size, width-patch_size, height, width])
                        # stop
                        extract_last_row = False
                        extract_finish = True
                        
                    else:
                        print(f"h:{height-patch_size} w:{w} ->  h':{height} w':{w+patch_size}")
                        patch_img = img.crop([height-patch_size, w, height, w+patch_size])
                        patch_label = label.crop([height-patch_size, w, height, w+patch_size])

                    if not os.path.exists(os.path.join(save_dest_for_image, img_dir)):
                        os.mkdir(os.path.join(save_dest_for_image, img_dir))
                    if not os.path.exists(os.path.join(save_dest_for_label, label_dir)):
                        os.mkdir(os.path.join(save_dest_for_label, label_dir))

                    patch_img_name = img_dir+'_'+str(height_count)+'_'+str(width_count)+'.png'
                    patch_label_name = label_dir+'_'+str(height_count)+'_'+str(width_count)+'.png'

                    patch_img.save(os.path.join(save_dest_for_image, img_dir, patch_img_name))
                    patch_label.save(os.path.join(save_dest_for_label, label_dir, patch_label_name))

                    # iter along w
                    w += patch_size
                    width_count += 1
                    print(height_count, width_count)
  
            else:
                # set row flag
                extract_row = True
                # initial row start
                w = 0
                width_count = 0
                # Extracting row
                while extract_row:
                    if w+patch_size > width:
                        print(f"h:{h} w:{width-patch_size} ->  h':{h+patch_size} w':{width}")
                        patch_img = img.crop([h, width-patch_size, h+patch_size, width])
                        patch_label = label.crop([h, width-patch_size, h+patch_size, width])
                        # stop
                        extract_row = False
                    else:
                        print(f"h:{h} w:{w} ->  h':{h+patch_size} w':{w+patch_size}")
                        patch_img = img.crop([h, w, h+patch_size, w+patch_size])
                        patch_label = label.crop([h, w, h+patch_size, w+patch_size])

                    if not os.path.exists(os.path.join(save_dest_for_image, img_dir)):
                        os.mkdir(os.path.join(save_dest_for_image, img_dir))
                    if not os.path.exists(os.path.join(save_dest_for_label, label_dir)):
                        os.mkdir(os.path.join(save_dest_for_label, label_dir))

                    patch_img_name = img_dir+'_'+str(height_count)+'_'+str(width_count)+'.png'
                    patch_label_name = label_dir+'_'+str(height_count)+'_'+str(width_count)+'.png'

                    patch_img.save(os.path.join(save_dest_for_image, img_dir, patch_img_name))
                    patch_label.save(os.path.join(save_dest_for_label, label_dir, patch_label_name))

                    # iter along w
                    w += patch_size
                    width_count += 1
                    print(height_count, width_count)

            # iter along h
            h += patch_size
            height_count += 1
            
        # extract patch end
        print(f"Image {img_dir} extracted finished !")

In [60]:
extract_test_patches(
    test_irrg_list, 
    test_irrg_label_list, 
    save_dest_for_image=save_test_image_dir, 
    save_dest_for_label=save_test_label_dir, 
    patch_size=512)

0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
6 12
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
7 10
7 11
7 12
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
8 10
8 11
8 12
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
9 9
9 10
9 11
9 12
10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
10 10
10 11
10 12
11 1
11 2
11 3
11 4
11 5
11 6
11 7
11 8
11 9
11 10
11 11
11 12
Image top_potsdam_2_13_IRRG extracted finished !
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
6 1
6 2
6

0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
6 12
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
7 10
7 11
7 12
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
8 10
8 11
8 12
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
9 9
9 10
9 11
9 12
10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
10 10
10 11
10 12
11 1
11 2
11 3
11 4
11 5
11 6
11 7
11 8
11 9
11 10
11 11
11 12
Image top_potsdam_6_15_IRRG extracted finished !
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
6 1
6 2
6

In [93]:
x=0
size = 512
num = 0

while x < 6000:
    num += 1
        
    t = x + size
    
    y = t-170
    print(f"Cor: {y} = {t}-170  Crop: [{x}, {t}]")
    
    x = y
    


print(f"extract num:{num}")

Cor: 342 = 512-170  Crop: [0, 512]
Cor: 684 = 854-170  Crop: [342, 854]
Cor: 1026 = 1196-170  Crop: [684, 1196]
Cor: 1368 = 1538-170  Crop: [1026, 1538]
Cor: 1710 = 1880-170  Crop: [1368, 1880]
Cor: 2052 = 2222-170  Crop: [1710, 2222]
Cor: 2394 = 2564-170  Crop: [2052, 2564]
Cor: 2736 = 2906-170  Crop: [2394, 2906]
Cor: 3078 = 3248-170  Crop: [2736, 3248]
Cor: 3420 = 3590-170  Crop: [3078, 3590]
Cor: 3762 = 3932-170  Crop: [3420, 3932]
Cor: 4104 = 4274-170  Crop: [3762, 4274]
Cor: 4446 = 4616-170  Crop: [4104, 4616]
Cor: 4788 = 4958-170  Crop: [4446, 4958]
Cor: 5130 = 5300-170  Crop: [4788, 5300]
Cor: 5472 = 5642-170  Crop: [5130, 5642]
Cor: 5814 = 5984-170  Crop: [5472, 5984]
Cor: 6156 = 6326-170  Crop: [5814, 6326]
extract num:18
